# RIS to CSV converter
Notes
1) The code below can be used to convert RIS files to CSV files
2) The RIS file can be generated using citationchaser.org website


In [5]:
import rispy
import pandas as pd

# Load and parse the RIS file
file_path = 'C:\\Users\\pedro\\Desktop\\Materials World\\references-2024-10-31.ris'
with open(file_path, 'r', encoding='utf-8') as file:
    entries = rispy.load(file)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(entries)
output_csv_path = 'C:\\Users\\pedro\\Desktop\\Materials World\\Review 1.csv'
df.to_csv(output_csv_path, index=False)

print(f"CSV file saved to {output_csv_path}")


CSV file saved to C:\Users\pedro\Desktop\Materials World\Review 1.csv


# Step 1 
How to fetch the reference list with the reference number, doi, title and author (with year)

it uses the crossref API


In [5]:
import requests
import pandas as pd

# Function to fetch references using the CrossRef API
def get_references_from_doi(doi):
    url = f"https://api.crossref.org/works/{doi}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        references = data.get('message', {}).get('reference', [])
        return references
    else:
        print(f"Error fetching data for DOI {doi}: {response.status_code}")
        return None

# Helper function to extract the first author's last name, handling various formats
def get_first_author_last_name(authors):
    if not authors:
        return "Unknown Author(s)"
    
    # Extract the first author
    author = authors[0]
    
    # Try splitting by comma, which may indicate "LastName, Initials"
    if "," in author:
        last_name = author.split(",")[0].strip()
    else:
        # If no comma, assume last name is the first word (handle cases like "Smith J.D." or "Jones, A.B.C.")
        parts = author.split()
        last_name = parts[0].strip()
    
    # Add "et al." if there are multiple authors
    return f"{last_name} et al." if len(authors) > 1 else last_name

# Helper function to extract necessary information for the table
def extract_reference_info(ref, order):
    # Extract the title
    title = ref.get('article-title', 'No Title')
    
    # Reference order number
    reference_number = order + 1  # Start from 1
    
    # Extract DOI if available
    ref_doi = ref.get('DOI', 'No DOI')
    
    # Get first author's last name with "et al." if multiple authors
    authors = ref.get('author', [])
    first_author_last_name = get_first_author_last_name(authors)
    
    # Extract the year of publication
    year = ref.get('year', 'No Year')
    
    # Return a dictionary for easy DataFrame conversion
    return {
        "Title": title,
        "Reference Number": reference_number,
        "DOI": ref_doi,
        "First Author": f"{first_author_last_name} ({year})"
    }

# Example DOI
doi = "10.1016/j.heliyon.2024.e25439"  # Replace with your DOI
references = get_references_from_doi(doi)

# Create a DataFrame from the extracted references information
if references:
    reference_data = [extract_reference_info(ref, i) for i, ref in enumerate(references)]
    df = pd.DataFrame(reference_data, columns=["Title", "Reference Number", "DOI", "First Author"])
    
    # Display the table
    print(df)
    
    # Optionally, save to CSV
    output_csv_path = 'C:\\Users\\pedro\\Desktop\\Materials World\\MFC Database\\Review 3 Jalili 2024\\Review 3 Jalili 2024 test 2.csv'
    df.to_csv(output_csv_path, index=False)
    print(f"References table saved to {output_csv_path}")
else:
    print("No references found.")


                                                 Title  Reference Number  \
0    Analysis of ammonia loss mechanisms in microbi...                 1   
1      Towards a science of climate and energy choices                 2   
2    Global energy perspectives to 2060–WEC's world...                 3   
3    Renewable energy and sustainable development: ...                 4   
4    A novel microbial fuel cell stack for continuo...                 5   
..                                                 ...               ...   
186  Microbial phenazine production enhances electr...               187   
187  Metabolites produced by Pseudomonas sp. enable...               188   
188  Anodic biofilms in microbial fuel cells harbor...               189   
189  Microfluidic microbial fuel cell: on-chip auto...               190   
190  Sediment microbial fuel cells as a barrier to ...               191   

                                DOI     First Author  
0                 10.1002/bit.21

# Step 3 - Fetch the Table info and values using tabula.org